# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

'0.4.1'

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [2]:
# !pip install gluonts

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [6]:
def deepar(data="m4_quarterly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(training_data=dataset.train, 
                                validation_data=dataset.test)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [7]:
%%time
res = deepar(data="m4_quarterly", seed=42, epochs=1000, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.29it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[0] Elapsed time 21.538 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.682600
0it [00:00, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 1230523
771it [02:32,  5.06it/s, validation_avg_epoch_loss=7.37]
INFO:root:Epoch[0] Elapsed time 152.420 seconds
INFO:root:Epoch[0] Evaluation metric 'validation_epoch_loss'=7.365896
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:22<00:00,  8.97it/s, avg_epoch_loss=7.17]
INFO:root:Epoch[1] Elapsed time 22.334 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.169172
769it [00:52, 14.63it/s, validation_avg_epoch_loss=7.24]
INFO:root:Epoch[1] Elapsed time 52.559 seconds
INFO:root:Epoch[1] Evaluation metric 'validation_epoch_loss'=7.23840

{'MASE': 1.43804045,
 'MSIS': 14.86544584,
 'epochs': 5,
 'sMAPE': 0.11262409,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.10476292,
 'wQuantileLoss[0.9]': 0.0653894}
Wall time: 34min 47s


In [ ]:
%%time
res = deepar(data="m4_yearly", seed=42, epochs=1000, batches=200)
pprint(res)

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html "DeepAR 09 - DeepAR experiments with gluonts".ipynb

In [ ]:
print("Done!")

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results